In [1]:
#从selenium导入浏览器驱动
from selenium import webdriver
#导入浏览器驱动设置选项
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
#导入网页解析库
from bs4 import BeautifulSoup
#导入时间库,利用time.time()防止爬虫访问过于频繁被禁止访问
import time
#导入pandas数据分析库,生成dataframe
import pandas as pd
import pickle

In [2]:
companies = ['阿里巴巴（中国）网络技术有限公司']

In [3]:
driver = webdriver.Chrome() #实例化一个浏览器
driver.get('https://www.qcc.com/')
print('请在打开的网页，扫码登录！')
time.sleep(1.238)
new_cookies = driver.get_cookies() #获取最新的cookies
pickle.dump( new_cookies, open("cookies.pkl","wb"))
driver.quit() #关闭浏览器

请在打开的网页，扫码登录！


In [4]:
for c in companies:
    #首先，初始化变量
    bs,bs1,bs2,partner= '','','',''
    part,employ,pos,hisem,hispos,time1 = [],[],[],[],[],[]
    
    '''若取消下方被注释的代码可静默运行浏览器，不会显示页面，仅在后台运行'''
    chrome_options=Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome()           #实例化一个谷歌浏览器对象
    driver.get("https://www.qcc.com/")   #浏览器打开企查查网站
    
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.get("https://www.qcc.com/")
    driver.refresh()                #自动刷新页面，请检查是否已经自动登录账号
    
#     try:
    time.sleep(2.273)

#     bs = BeautifulSoup(driver.page_source,'html.parser')  #将加载好的网页用BeautifulSoup解析成文本
#     print(bs)
    driver.find_element(By.XPATH, "//input[contains(@id,'searchKey')]").send_keys(c)
    driver.find_element(By.XPATH, "//button[@class='btn btn-primary']").click()
    time.sleep(3)
    bs = BeautifulSoup(driver.page_source,'html.parser')  #将加载好的网页用BeautifulSoup解析成文本
    
    wait = WebDriverWait(driver, 3)
    links = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href,'https://www.qcc.com/firm/')]")))
    driver.execute_script("arguments[0].target='_self';", links[0])
    driver.find_element(By.XPATH, "//a[contains(@href,'https://www.qcc.com/firm/')]").click()
    time.sleep(3)
    bs = BeautifulSoup(driver.page_source,'html.parser')  #将加载好的网页用BeautifulSoup解析成文本
    
    html_table = bs.find("table")
    str_table = str(html_table)
    print(str_table)
    
    # 成立年份
    str_table_year = re.compile(r'([1|2]\d\d\d)[-](0[1-9]|1[012])[-](0[1-9]|[12][0-9]|3[01])')
    iterator = str_table_year.findall(str_table)
    if len(list(iterator)) > 0:
         print('成立年份:', list(iterator)[0][0])
    elif "年" in str_table:
        str_table_year = re.compile(r'([1|2]\d\d\d)[年](\d+)[月](\d+)[日]')
        iterator = str_table_year.findall(str_table)
        print('成立年份:', list(iterator)[0][0])
    
    # 所属地区
    headquarters = {
        '开曼群岛': 'Cayman Islands',
        '開曼群島':'Cayman Islands',
        '英属维尔京群岛': 'British Virgin Islands'
    }
    with open('chinese_cities.csv', newline='') as csvfile:
        headquarter_reader = csv.reader(csvfile, delimiter=',')
        for en_city, cn_city, en_province in headquarter_reader:
            if cn_city in ['香港', '澳门', '澳門', '北京市', '重庆市', '上海市', '天津市']:
                headquarters[cn_city] = "{}, China".format(en_city)
            else:
                headquarters[cn_city] = "{}, {}, China".format(en_city, en_province)
    print(companies)
    en_headquarter = [headquarter for cn_city, headquarter in headquarters.items() if cn_city in str_table]
    if '非香港' in str_table:
        en_headquarter.remove('Hong Kong, China')
        print('Headquarter is N/A')
    if 'Cayman Islands' in en_headquarter:
        print('Put in incorporated catergory')
    else:
        print(en_headquarter)